In [1]:
import os
from pathlib import Path
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine

In [2]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [3]:
def search_costo_ventas_func(mes: int, tolerance: list) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    if type(tolerance) != list or len(tolerance) != 2:
        raise Exception (f'La tolerancia debe ser una lista de sólo dos elementos')

    if type(tolerance[0]) != int or type(tolerance[1]) != int:
        raise Exception (f'Los elementos de la lista de tolerancia deben ser sólo números tipo enteros')
    
    param_dia_hr_ini = pd.Timestamp(2024, mes, 1) + pd.Timedelta(hours=7)
    param_dia_hr_fin = pd.Timestamp(2024, mes+1, 1) + pd.Timedelta(hours=7) - pd.Timedelta(seconds= 1)
    
    param_dia_hr_ini_tolerance = param_dia_hr_ini - pd.Timedelta(days = tolerance[0])
    param_dia_hr_fin_tolerance = param_dia_hr_fin + pd.Timedelta(days = tolerance[1])
    
    search_costo_ventas = [
        "&", "&",
            ("state", "in", ["purchase", "done"]),
            ("date_approve", ">=", param_dia_hr_ini_tolerance.strftime('%Y-%m-%d %H:%M:%S')),
            ("date_approve", "<=", param_dia_hr_fin_tolerance.strftime('%Y-%m-%d %H:%M:%S')),
        ]

    return search_costo_ventas

In [4]:
def mes_to_string_func(mes: int) -> str:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    match mes:
        case 1:
            mes_to_string = 'enero'
        case 2:
            mes_to_string = 'febrero'
        case 3:
            mes_to_string = 'marzo'
        case 4:
            mes_to_string = 'abril'
        case 5:
            mes_to_string = 'mayo'
        case 6:
            mes_to_string = 'junio'
        case 7:
            mes_to_string = 'julio'
        case 8:
            mes_to_string = 'agosto'
        case 9:
            mes_to_string = 'septiembre'
        case 10:
            mes_to_string = 'octubre'
        case 11:
            mes_to_string = 'noviembre'
        case 12:
            mes_to_string = 'diciembre'

    return mes_to_string

In [5]:
def get_dfs_from_database(db_mode: str, mes_to_string:str) -> list[pd.DataFrame]:
    
    if db_mode.lower() == 'local':
    
        db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/comisiones.db'))
        engine = create_engine(f'sqlite:///{db_file_path_str}')

        with engine.connect() as conn, conn.begin():
            ventas = pd.read_sql_table(f'ventas_{mes_to_string}', conn, dtype_backend='numpy_nullable')
            try:
                ultimo_costo = pd.read_sql_table(f'ultimo_costo_{mes_to_string}', conn, dtype_backend='numpy_nullable')
            except:
                ultimo_costo = None

        engine.dispose()
    

    elif db_mode.lower() != 'local':
        raise Exception (f'Sólo existe la base de datos "Local"')


    return ventas, ultimo_costo

In [6]:
def _get_df_from_excel(file_name:str, file_sheet:str, file_location:str) -> pd.DataFrame:

    file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{file_location}/{file_name}'))
    df = pd.read_excel(file_path_str, sheet_name=file_sheet, dtype_backend='numpy_nullable')

    return df

In [7]:
def prov_locales_df_from_excel() -> list[pd.DataFrame]:
    
    file_name = 'proveedores_oficiales.xlsx'
    file_sheet = 'Hoja1'
    file_location = 'compras'

    df = _get_df_from_excel(file_name, file_sheet, file_location)

    prov_oficiales = df.loc[df['oficial'] == 1][['partner_id', 'partner_name']]
    prov_locales = df.loc[df['oficial'] == 0][['partner_id', 'partner_name']]

    return prov_oficiales, prov_locales

In [8]:
def ultimo_costo_sae_df_from_excel() -> list[pd.DataFrame]:
    
    file_name = 'ultimo_costo_sae.xlsx'
    file_sheet = 'Hoja1'
    file_location = 'costo_ventas'

    df = _get_df_from_excel(file_name, file_sheet, file_location)

    return df

In [9]:
def calificados_fase_c_df_from_excel() -> list[pd.DataFrame]:

    file_name = 'calificados_fase_c.xlsx'
    file_sheet = 'Hoja1'
    file_location = 'costo_ventas'

    df = _get_df_from_excel(file_name, file_sheet, file_location)

    # Check que verifica que todas las líneas de compra estén calificadas
    if pd.NA in df['calificacion'].unique():
        print('¡Cuidado!, tienes líneas de compras "line_id" sin calificar')
        return None
    
    df['calificacion'] = df['calificacion'].str.upper()
    calificados_fase_c_OK = df.loc[df['calificacion'] == 'OK'] # Líneas 'line_id' que tienen un match con 'fact_line_id'
    calificados_fase_c_B = df.loc[df['calificacion'] == 'B'] # Líneas para regresar a la fase B y que el algoritmo las catalogue.
    calificados_fase_c_C = df.loc[df['calificacion'] == 'C'] # Líneas para regresar a la fase C y que el humano las catalogue.
    calificados_fase_c_D = df.loc[df['calificacion'] == 'D'] # Líneas para que el algoritmo fase D (final) las catalogue.
    calificados_fase_c_E = df.loc[df['calificacion'] == 'E'] # Líneas para eliminar, la observación tiene el por qué.


    return calificados_fase_c_OK, calificados_fase_c_B, calificados_fase_c_C, calificados_fase_c_D, calificados_fase_c_E

In [10]:
def repetidos_match_1ro_df_from_excel(mes:int) -> list[pd.DataFrame]:

    file_name = f'{mes}_costo_ventas.xlsx'
    file_sheet = 'repetidos_match_1ro'
    file_location = 'costo_ventas'

    df = _get_df_from_excel(file_name, file_sheet, file_location)

    return df

In [11]:
def api_call_purchase_doc_func(api_params: dict, search_costo_ventas: list[str] ) -> list[int, dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    purchase_doc_fields = [
                    'name',
                    'state',
                    'partner_id',
                    'partner_ref',
                    'date_approve',
                    'x_fecha_factura',
                    'user_id',
                    'create_uid'
                    ]

    purchase_doc_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [search_costo_ventas])
    purchase_doc_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [purchase_doc_ids], {'fields': purchase_doc_fields})
    
    return purchase_doc_ids, purchase_doc_json

In [12]:
def purchase_doc_func(purchase_doc_json: list[dict], prov_oficiales:pd.DataFrame) -> pd.DataFrame:

    purchase_doc_data = []

    for compra in purchase_doc_json:
        new = {}
        new['order_id'] = compra['id']
        new['order_name'] = compra['name']
        new['order_state'] = compra['state']
        new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
        new['partner_id'] = compra['partner_id'][0]
        new['partner_name'] = compra['partner_id'][1]
        new['partner_fact_ref'] = compra['partner_ref']
        new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
        new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
        new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

        purchase_doc_data.append(new)

    compras_doc = pd.DataFrame(purchase_doc_data)
    compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
    compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')
    compras_doc['oficial'] = compras_doc['partner_id'].isin(prov_oficiales['partner_id'])

    return compras_doc


In [13]:
def api_call_purchase_line_func(api_params: dict, purchase_doc_ids: list[int]) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    purchase_line_fields = [
                        'order_id',
                        'date_approve',
                        'partner_id',
                        'product_id',
                        'product_qty',
                        'price_unit_discounted'
                        ]

    purchase_line_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", purchase_doc_ids)]])
    purchase_line_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [purchase_line_ids], {'fields': purchase_line_fields})
    
    return purchase_line_json

In [14]:
def purchase_line_func(purchase_line_json: list[dict]) -> pd.DataFrame:
    
    purchase_line_data = []

    for line in purchase_line_json:
        new = {}
        new['line_id'] = line['id']
        new['order_id'] = line['order_id'][0]
        new['order_name'] = line['order_id'][1]
        new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
        new['partner_id'] = line['partner_id'][0]
        new['partner_name'] = line['partner_id'][1]
        new['product_id_pp'] = line['product_id'][0]
        new['product_name'] = line['product_id'][1]
        new['product_qty'] = line['product_qty']
        new['product_cost'] = line['price_unit_discounted']
        
        purchase_line_data.append(new)

    compras_line = pd.DataFrame(purchase_line_data)
    compras_line['order_date'] = pd.to_datetime(compras_line['order_date'], format='%Y-%m-%d %H:%M:%S')

    return compras_line

In [15]:
def compras_func(compras_doc:pd.DataFrame, compras_line:pd.DataFrame, ultimo_costo:pd.DataFrame) -> pd.DataFrame:
    
    compras_odoo = pd.merge(
                    compras_line,
                    compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'
                )

    compras_odoo['order_date'] = compras_odoo['order_date'].dt.normalize()

    cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
    compras_odoo[cols_to_Int64] = compras_odoo[cols_to_Int64].astype('Int64')

    compras_odoo['product_qty'] = compras_odoo['product_qty'].astype('Float64')
    compras_odoo['vendedora'] = compras_odoo['vendedora'].convert_dtypes()

    compras = pd.concat([compras_odoo, ultimo_costo])

    return compras, compras_odoo

# Pruebas

In [16]:
mes = 1

api_params = api_params_func()
search_costo_ventas = search_costo_ventas_func(mes, tolerance=[0,0])
mes_to_string = mes_to_string_func(mes)

ventas, ultimo_costo = get_dfs_from_database('local', mes_to_string)

prov_oficiales, prov_locales = prov_locales_df_from_excel()

if not ultimo_costo:
    ultimo_costo = ultimo_costo_sae_df_from_excel()
calificados_fase_c_OK, calificados_fase_c_B, calificados_fase_c_C, calificados_fase_c_D, calificados_fase_c_E = calificados_fase_c_df_from_excel()

purchase_doc_ids, purchase_doc_json = api_call_purchase_doc_func(api_params, search_costo_ventas)
purchase_line_json = api_call_purchase_line_func(api_params, purchase_doc_ids)

compras_doc = purchase_doc_func(purchase_doc_json, prov_oficiales)
compras_line = purchase_line_func(purchase_line_json)
compras, compras_odoo = compras_func(compras_doc, compras_line, ultimo_costo)


# Checks

In [17]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(pd.concat([prov_locales, prov_oficiales])['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')

Todo correcto con check1


In [18]:
from IPython.display import display

check_costo_venta = (
    pd.merge_asof(
        left = ventas.sort_values('invoice_date'),
        right = compras.sort_values('order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'order_date', 

        direction = 'backward')
)

check2 = check_costo_venta[check_costo_venta['product_cost'].isna()][['product_id', 'product_name_x']]

print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 0 renglones sin costo de la venta.
Todo correcto con check2


# Desarrollo del código

In [19]:
cols_vent = ['fact_doc_id',
 'name',
 'invoice_date',
 'fact_line_id',
 'quantity',
 'price_unit',
 'salesperson_name',
 'product_id',
 'product_name']

In [95]:
repetidos_match_1ro = repetidos_match_1ro_df_from_excel(mes)


lista_capturistas = [
       'Elsa Ivette Diaz Leyva',
       'Alexa Yadira Mazariegos Zunun',
       'Dulce Guadalupe Pedroza Valenzuela',
       'Mariana Araceli Carvajal Flores',
       'Rosario Martinez Zarate'
]

# Compras especiales que no son de proveedores oficiales
compras_especiales = compras_odoo.loc[
            (~compras_odoo['vendedora'].isin(lista_capturistas))
            & (~compras_odoo['partner_id'].isin(prov_oficiales['partner_id']))
            ]

compras_especiales_after_repetidos = compras_especiales.loc[
            ~compras_especiales['line_id'].isin(repetidos_match_1ro['line_id'])
            ]

ventas_after_repetidos = ventas.loc[
            ~ventas['fact_line_id'].isin(repetidos_match_1ro['fact_line_id'])
            ]

In [96]:
match_1_merge = (
    pd.merge_asof(
        left = compras_especiales_after_repetidos.sort_values('order_date'), 
        right = ventas_after_repetidos.sort_values('invoice_date'),
        
        left_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        right_by = ['product_id', 'salesperson_name', 'quantity'], 
        
        left_on = 'order_date', 
        right_on = 'invoice_date', 

        direction = 'nearest',
        tolerance = pd.Timedelta(days=3))
)

In [97]:
ids_ventas_repetidas_por_corregir = match_1_merge.loc[(~match_1_merge['fact_line_id'].isna()) & (match_1_merge['fact_line_id'].duplicated()), 'fact_line_id']

if not ids_ventas_repetidas_por_corregir.empty:

    cols = [
        'line_id', 'fact_line_id', '.',
        'order_name', 'order_date', 'partner_name_x', 'partner_fact_ref', 'partner_fact_date',  'product_qty',
        'product_cost', 'vendedora', 'product_id_pp', 'product_name_x', '.',
        'fact_doc_id', 'name', 'invoice_date', 'partner_name_y', 'quantity', 'salesperson_name', 'price_unit', 'discount',]


    
    repetidos_match_1ro_por_corregir = match_1_merge.loc[match_1_merge['fact_line_id'].isin(ids_ventas_repetidas_por_corregir)].reset_index()
    repetidos_match_1ro_por_corregir.loc[:,'.'] = pd.NA

    archivo = 'repetidos_match_1ro_por_corregir.xlsx'
    desktop_path = Path.home().joinpath(f'Desktop/{archivo}')
    repetidos_match_1ro_por_corregir[cols].to_excel(desktop_path)
    print(f'¡Alto! tienes "fact_line_id" repetidas. \nSe generó un archivo excel en tu escritorio llamado "{archivo}" para que corrijas la información.')

else:
    print('No hay ningún "fact_line_id" repetido en el primer match. Puedes continuar...')
    

No hay ningún "fact_line_id" repetido en el primer match. Puedes continuar...


In [98]:
# Match resultantes del 1er match.
match_1 = match_1_merge.loc[~match_1_merge['fact_line_id'].isna(), ['line_id', 'fact_line_id']]

# Después de correr el 1er match, las ventas restantes
ventas_after_match_1 = ventas_after_repetidos.loc[
                                        ~ventas_after_repetidos['fact_line_id'].isin(match_1['fact_line_id'])
                                    ]

# Después de correr el 1er match, las compras que tienen un product_id que sí existe en las ventas restantes.
compras_especiales_after_match_1 = compras_especiales_after_repetidos.loc[
                                                        (~compras_especiales_after_repetidos['line_id'].isin(match_1['line_id']))
                                                        & (compras_especiales_after_repetidos['product_id_pp'].isin(ventas_after_match_1['product_id']))
                                                    ]

# Resto de las compras especiales. Estan no tienen un product_id que existe en las ventas restantes y no se pueden merchar.
compras_especiales_sin_ventas = compras_especiales_after_repetidos.loc[
                                                        (~compras_especiales_after_repetidos['line_id'].isin(match_1['line_id']))
                                                        & (~compras_especiales_after_repetidos['product_id_pp'].isin(ventas_after_match_1['product_id']))
                                                    ]

In [99]:
match_2 = pd.DataFrame([], columns=['line_id','fact_line_id'])

for i in range(len(compras_especiales_after_match_1)):
    
    linea_compra = compras_especiales_after_match_1.sort_values('order_date').iloc[i]

    mini_df = ventas_after_match_1.loc[
                (~ventas_after_match_1['fact_line_id'].isin(match_2['fact_line_id']))
                & (ventas_after_match_1['salesperson_name'] == linea_compra['vendedora'])
                & (ventas_after_match_1['product_id'] == linea_compra['product_id_pp'])
                # & (ventas_after_match_1['invoice_date'] >= linea_compra['order_date'] - pd.Timedelta(days=5))
                # & (ventas_after_match_1['invoice_date'] <= linea_compra['order_date'] + pd.Timedelta(days=5))
            ]
    
    if not mini_df.empty:
        df_copia = mini_df.copy()
        df_copia['diff'] = abs(df_copia['invoice_date'] - linea_compra['order_date'])
        df_sort = df_copia.sort_values('diff').reset_index()
        df_sort['cumsum'] = df_sort['quantity'].cumsum()
        index = df_sort[df_sort['cumsum'] == linea_compra['product_qty']].index[0] if not df_sort[df_sort['cumsum'] == linea_compra['product_qty']].index.empty else None
        if index != None:
            df_sort.loc[:index , 'line_id'] = linea_compra['line_id']

            if match_2.empty:
                match_2 = df_sort.loc[~df_sort['line_id'].isna(), ['line_id','fact_line_id']].copy()
            else:
                match_2 = pd.concat([
                            match_2, 
                            df_sort.loc[~df_sort['line_id'].isna(), ['line_id','fact_line_id']],
                        ])
            
        else:    
            df_sort.loc[:index , 'line_id'] = pd.NA

In [100]:
# Después de correr el 2do match, las ventas restantes
ventas_after_match_2 = ventas_after_match_1.loc[
                                        ~ventas_after_match_1['fact_line_id'].isin(match_2['fact_line_id'])
                                    ]

# Después de correr el 2do match, las compras restantes.
compras_especiales_after_match_2 = compras_especiales_after_match_1.loc[
                                            ~compras_especiales_after_match_1['line_id'].isin(match_2['line_id'])
                                        ]

In [66]:
match_3 = (
    pd.merge_asof(
        left = compras_especiales_after_match_2.sort_values('order_date'), 
        right = ventas_after_match_2.sort_values('invoice_date'),
        
        left_by = ['product_id_pp', 'vendedora', 'product_qty'], 
        right_by = ['product_id', 'salesperson_name', 'quantity'], 
        
        left_on = 'order_date', 
        right_on = 'invoice_date', 

        direction = 'nearest',
        tolerance = pd.Timedelta(days=3))
)

In [68]:
match_3[~match_3['fact_line_id'].isna()]

,line_id,order_id,order_name,order_date,partner_id_x,partner_name_x,product_id_pp,product_name_x,product_qty,product_cost,...,warehouse,fact_line_id,product_id,product_name_y,prod_codigo,prod_linea,quantity,price_unit,discount,price_subtotal


In [ ]:
match_4 = pd.DataFrame([], columns=['line_id','fact_line_id'])

for i in range(len(compras_especiales_after_match_2)):
    
    linea_compra = compras_especiales_after_match_2.sort_values('order_date').iloc[i]

    mini_df = ventas_after_match_2.loc[
                (~ventas_after_match_2['fact_line_id'].isin(match_4['fact_line_id']))
                # & (ventas_after_match_2['salesperson_name'] == linea_compra['vendedora'])
                & (ventas_after_match_2['product_id'] == linea_compra['product_id_pp'])
                # & (ventas_after_match_2['invoice_date'] >= linea_compra['order_date'] - pd.Timedelta(days=5))
                # & (ventas_after_match_2['invoice_date'] <= linea_compra['order_date'] + pd.Timedelta(days=5))
            ]
    
    print(f"-----> línea de compra {linea_compra['line_id']}")
    print(f"Compra:")
    display(compras_especiales_after_match_2[compras_especiales_after_match_2['line_id'] == linea_compra['line_id']])
    print(f"Ventas mini_df")
    display(mini_df[cols_vent])
    print(f"\n\n")

    # if not mini_df.empty:
    #     df_copia = mini_df.copy()
    #     df_copia['diff'] = abs(df_copia['invoice_date'] - linea_compra['order_date'])
    #     df_sort = df_copia.sort_values('diff').reset_index()
    #     df_sort['cumsum'] = df_sort['quantity'].cumsum()
    #     index = df_sort[df_sort['cumsum'] == linea_compra['product_qty']].index[0] if not df_sort[df_sort['cumsum'] == linea_compra['product_qty']].index.empty else None
    #     if index != None:
    #         df_sort.loc[:index , 'line_id'] = linea_compra['line_id']

    #         if match_4.empty:
    #             match_4 = df_sort.loc[~df_sort['line_id'].isna(), ['line_id','fact_line_id']].copy()
    #         else:
    #             match_4 = pd.concat([
    #                         match_4, 
    #                         df_sort.loc[~df_sort['line_id'].isna(), ['line_id','fact_line_id']],
    #                     ])
            
    #     else:    
    #         df_sort.loc[:index , 'line_id'] = pd.NA

In [122]:
ids_prod = compras_especiales_after_match_2['product_id_pp'].unique()

for prod in ids_prod:
    
    mini_df = ventas_after_match_2.loc[
                (ventas_after_match_2['product_id'] == prod)
            ]
    
    print(f"-----> product_id {prod}")
    print(f"Compra:")
    display(compras_especiales_after_match_2[compras_especiales_after_match_2['product_id_pp'] == prod])
    print(f"Ventas mini_df")
    display(mini_df[cols_vent])
    print(f"\n\n")

-----> product_id 13786
Compra:


,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_cost,partner_fact_ref,partner_fact_date,capturista,vendedora
0,1812,498,P00487 (SJCFCO 2727),2024-01-31,5326,Madereria El Pino De Los Cabos,13786,"[13538] Tabla Americana 1"" X 12"" X 16' *13538*",1.0,537.93,SJCFCO 2727,NaT,<NA>,Mayra Angelica Parada Manjarrez
1157,541,197,P00186 (A 103781),2024-01-13,5352,Luz Raquel Rodriguez Montoya,13786,"[13538] Tabla Americana 1"" X 12"" X 16' *13538*",5.0,524.14,A 103781,NaT,<NA>,Mayra Angelica Parada Manjarrez


Ventas mini_df


,fact_doc_id,name,invoice_date,fact_line_id,quantity,price_unit,salesperson_name,product_id,product_name
3644,5961,F1-VS/2024/00148,2024-01-12,21835,5.0,806.9,Yamilet Blanco Salas,13786,"Tabla Americana 1"" X 12"" X 16' *13538*"





-----> product_id 28276
Compra:


...

,fact_doc_id,name,invoice_date,fact_line_id,quantity,price_unit,salesperson_name,product_id,product_name
849,1579,F1-VS/2024/00028,2024-01-05,5801,125.0,81.36,Irma Carvajal Flores,10856,Tope Ph Media Luna #54-C #1464 *528*
2350,3911,F1-VS/2024/00095,2024-01-09,14241,100.0,81.36,Irma Carvajal Flores,10856,Tope Ph Media Luna #54-C #1464 *528*





-----> product_id 14773
Compra:


,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id_pp,product_name,product_qty,product_cost,partner_fact_ref,partner_fact_date,capturista,vendedora
1617,54,40,P00033 (2000005232684875),2024-01-03,5128,Mercado Libre,14773,[1375] Disco Abrasivo Desbaste Metal 4-1/2x7/8...,25.0,25.14,2000005232684875,NaT,<NA>,Yolanda Alejandra Rodriguez González


Ventas mini_df


,fact_doc_id,name,invoice_date,fact_line_id,quantity,price_unit,salesperson_name,product_id,product_name
708,1374,F2-CC/2024/00177,2024-01-05,5084,20.0,71.5,Yolanda Alejandra Rodriguez González,14773,Disco Abrasivo Desbaste Metal 4-1/2x7/8 B-4426...
6803,10813,F2-CC/2024/01390,2024-01-22,40206,1.0,71.5,Patricia Flores Pantaleón,14773,Disco Abrasivo Desbaste Metal 4-1/2x7/8 B-4426...
